In [25]:
import os
import json
import pandas as pd
from pandas.io.json import json_normalize
import requests
from datetime import datetime

from google.cloud import storage


In [26]:
os.getcwd()
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/c/users/leand/AirflowHome/shark-project-322205-f223d86d85bb.json'

In [27]:
data_json=[]

In [28]:
symbols=['NFLX','MSFT','GOOGL','FB','AMZN','BABA','TSLA']
    
for val in symbols:
    r = requests.get('https://finnhub.io/api/v1/news-sentiment?symbol='+str(val)+'&token=c48am02ad3icscig0me0')
    data_json.append(r.json())
    df = json_normalize(data_json)
    

/tmp/ipykernel_19426/3927414560.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(data_json)


In [29]:

df_cols=df.iloc[:,[1,3,5,6,2,7]]
df_cols.columns=['NewsScore','Symbol','Buzz','WeekAvgBuzz','BearSentiment','BullSentiment']

df_cols=df_cols[['Symbol','NewsScore','Buzz','WeekAvgBuzz','BearSentiment','BullSentiment']]

df_cols['Date']=datetime.now().strftime("%Y-%m-%d")
df_final=df_cols.sort_values(by=['BullSentiment'],ascending=False).reset_index()
df_final=df_cols

In [30]:
df_final.to_csv("/c/Users/leand/AirflowHome/dags/stocks.csv")

In [31]:
df_final

,Symbol,NewsScore,Buzz,WeekAvgBuzz,BearSentiment,BullSentiment,Date
0,NFLX,0.6690,0.4769,130.00,0.5495,0.0000,2021-08-12
1,MSFT,0.6690,0.6894,197.25,0.5495,0.1667,2021-08-12
2,GOOGL,0.6690,0.6353,250.25,0.5495,0.1429,2021-08-12
3,FB,0.6625,0.6608,230.00,0.5458,0.5218,2021-08-12
4,AMZN,0.6489,0.7607,336.50,0.5465,0.2188,2021-08-12
5,BABA,0.6519,1.1443,93.50,0.5462,0.6191,2021-08-12
6,TSLA,0.6561,0.6302,201.50,0.5465,0.1667,2021-08-12


In [32]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
   
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        "File {} uploaded to {}.".format(
            source_file_name, destination_blob_name
        )
    )


upload_blob('stock_data_bucket1', '/c/users/leand/AirflowHome/dags/stocks.csv', 'stock_data.csv')

File /c/users/leand/AirflowHome/dags/stocks.csv uploaded to stock_data.csv.
